In [2]:
import ee

# Initialize the Earth Engine module.
ee.Initialize()

# Print metadata for a DEM dataset.
print(ee.Image('USGS/SRTMGL1_003').getInfo())

{'type': 'Image', 'bands': [{'id': 'elevation', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [1296001, 417601], 'crs': 'EPSG:4326', 'crs_transform': [0.0002777777777777778, 0, -180.0001388888889, 0, -0.0002777777777777778, 60.00013888888889]}], 'id': 'USGS/SRTMGL1_003', 'version': 1605362602494378, 'properties': {'system:visualization_0_min': '0.0', 'type_name': 'Image', 'thumb': 'https://mw1.google.com/ges/dd/images/SRTM90_V4_thumb.png', 'description': '<p>The Shuttle Radar Topography Mission (SRTM, see <a href="http://onlinelibrary.wiley.com/doi/10.1029/2005RG000183/full">Farr\net al. 2007</a>)\ndigital elevation data is an international research effort that\nobtained digital elevation models on a near-global scale. This\nSRTM V3 product (SRTM Plus) is provided by NASA JPL\nat a resolution of 1 arc-second (approximately 30m).</p><p>This dataset has undergone a void-filling process using open-source data\n(ASTER GDEM2, GMTED2010, a

In [2]:
# Import the Sentinel 1 ground collection (dB).
s1_grd = ee.ImageCollection('COPERNICUS/S1_GRD')

# Import the MODIS land cover collection.
#lc = ee.ImageCollection('MODIS/006/MCD12Q1')

# Import the MODIS land surface temperature collection.
#lst = ee.ImageCollection('MODIS/006/MOD11A1')

# Import the USGS ground elevation image.
#elv = ee.Image('USGS/SRTMGL1_003')

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
%matplotlib inline

# Import the Folium library.
import folium

In [3]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name = name,
        overlay = True,
        control = True
      ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [4]:
# specify extend
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              13.43902587890625,
              46.922131240914
            ],
            [
              13.993835449218748,
              46.922131240914
            ],
            [
              13.993835449218748,
              47.19997723868725
            ],
            [
              13.43902587890625,
              47.19997723868725
            ],
            [
              13.43902587890625,
              46.922131240914
            ]
          ]
        ]
      }
    }
  ]
}

In [5]:
# get coordinates
coords = geoJSON['features'][0]['geometry']['coordinates']
# create ee.Geometry()
aoi = ee.Geometry.Polygon(coords)

In [6]:
# Next, we filter the S1 archive to get an image over the aoi acquired sometime in August, 2020. 
# Any old image will do fine, so we won't bother to specify the orbit number or whether we want
# the ASCENDING or DESCENDING node
ffa_db = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD') 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2018-09-06'), ee.Date('2018-09-17')) 
                       .first() 
                       .clip(aoi))

In [7]:
# list band names
ffa_db.bandNames().getInfo()

['VV', 'VH', 'angle']

In [10]:
ffa_db.getInfo()

{'type': 'Image',
 'bands': [{'id': 'VV',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [4265, 3157],
   'origin': [19926, 10663],
   'crs': 'EPSG:32633',
   'crs_transform': [10, 0, 181889.7512195385, 0, -10, 5335200.730393572]},
  {'id': 'VH',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [4265, 3157],
   'origin': [19926, 10663],
   'crs': 'EPSG:32633',
   'crs_transform': [10, 0, 181889.7512195385, 0, -10, 5335200.730393572]},
  {'id': 'angle',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [5, 2],
   'origin': [2, 3],
   'crs': 'EPSG:32633',
   'crs_transform': [-13013.815310757665,
    -4434.552439636493,
    478739.8564611307,
    2614.558436948806,
    -19876.93262932729,
    5281583.291885802]}],
 'id': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20180909T051832_20180909T051857_023617_0292CA_B17A',
 'version': 1622186246359140.0,
 'properties': {'GRD_Post_Processing_start': 1536478963150,


In [8]:
url = ffa_db.select('VV').getThumbURL({'min': -20, 'max': 0})
disp.Image(url=url, width=800)

In [26]:
# download via google drive
task = ee.batch.Export.image.toDrive(image=ffa_db.select('VV'),
                                     description='sentinel1_20180909_tamsweg',
                                     region=aoi_tams,
                                     fileNamePrefix='export_tams',
                                     crs='EPSG:31287',
                                     fileFormat='GeoTIFF')
task.start()


In [31]:
task.status()

{'state': 'COMPLETED',
 'description': 'sentinel1_20180909_tamsweg',
 'creation_timestamp_ms': 1622190103083,
 'update_timestamp_ms': 1622190482450,
 'start_timestamp_ms': 1622190121692,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://drive.google.com/'],
 'attempt': 1,
 'id': 'AADZM44VFESWKJWPDBUVDIZF',
 'name': 'projects/earthengine-legacy/operations/AADZM44VFESWKJWPDBUVDIZF'}

In [25]:
# dowload via url link
link = ffa_db.select('VV').getDownloadURL({
    'crs': 'EPSG:31287',
    'fileFormat': 'GeoTIFF',
    'region': aoi_tams})
print(link)

EEException: Pixel grid dimensions (10573x7744) must be less than or equal to 10000.

In [ ]:
# get coordinates
coords = geoJSON['features'][0]['geometry']['coordinates']
# create ee.Geometry()
aoi = ee.Geometry.Polygon(coords)

In [14]:
coords

[[[13.43902587890625, 46.922131240914],
  [13.993835449218748, 46.922131240914],
  [13.993835449218748, 47.19997723868725],
  [13.43902587890625, 47.19997723868725],
  [13.43902587890625, 46.922131240914]]]

In [19]:
13.43902587890625 + ((13.993835449218748 - 13.43902587890625) * 0.25)

13.577728271484375

In [20]:
46.922131240914 + ((47.19997723868725 - 46.922131240914) * 0.25)

46.99159274035731

In [21]:
coords2 = [[[13.43902587890625, 46.922131240914],
          [13.577728271484375, 46.922131240914],
          [13.577728271484375, 46.99159274035731],
          [13.43902587890625, 46.99159274035731],
          [13.43902587890625, 46.922131240914]]]

In [23]:
# create ee.Geometry()
aoi_tams = ee.Geometry.Polygon(coords2)